In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.svm import SVC
from collections import Counter

In [3]:
columns = [
    "PassengerId", "Pclass", "Name", "Sex", "Age", "SibSp", "Parch", "Ticket", "Fare", "Cabin", "Embarked"
]

target = ["Embarked"]

In [5]:
file_path= Path("test.csv")
df= pd.read_csv(file_path)

df=df.dropna(axis=1, how='any')
Survived = dict.fromkeys(['S'], 'Survived')  
perished=dict.fromkeys(['C'], 'Perished')
missing= dict.fromkeys(['Q'],'Missing')
df = df.replace(Survived)
df=df.replace(perished)
df=df.replace(missing)

df.tail()

,PassengerId,Pclass,Name,Sex,SibSp,Parch,Ticket,Embarked
413,1305,3,"Spector, Mr. Woolf",male,0,0,A.5. 3236,Survived
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,0,0,PC 17758,Perished
415,1307,3,"Saether, Mr. Simon Sivertsen",male,0,0,SOTON/O.Q. 3101262,Survived
416,1308,3,"Ware, Mr. Frederick",male,0,0,359309,Survived
417,1309,3,"Peter, Master. Michael J",male,1,1,2668,Perished


# SPLIT DATA

In [34]:
# Create our features
X = df.drop(columns='Embarked')
X=pd.get_dummies(X)


# Create our target
y = df['Embarked']

In [35]:
X.isnull().mean()*100

PassengerId                                        0.0
Pclass                                             0.0
SibSp                                              0.0
Parch                                              0.0
Name_Abbott, Master. Eugene Joseph                 0.0
Name_Abelseth, Miss. Karen Marie                   0.0
Name_Abelseth, Mr. Olaus Jorgensen                 0.0
Name_Abrahamsson, Mr. Abraham August Johannes      0.0
Name_Abrahim, Mrs. Joseph (Sophie Halaut Easu)     0.0
Name_Aks, Master. Philip Frank                     0.0
Name_Aldworth, Mr. Charles Augustus                0.0
Name_Allison, Mr. Hudson Joshua Creighton          0.0
Name_Andersen, Mr. Albert Karvin                   0.0
Name_Andersson, Miss. Ida Augusta Margareta        0.0
Name_Andersson, Mr. Johan Samuel                   0.0
Name_Andrew, Mr. Frank Thomas                      0.0
Name_Angheloff, Mr. Minko                          0.0
Name_Angle, Mr. William A                          0.0
Name_Arons

In [36]:
X.describe()

,PassengerId,Pclass,SibSp,Parch,"Name_Abbott, Master. Eugene Joseph","Name_Abelseth, Miss. Karen Marie","Name_Abelseth, Mr. Olaus Jorgensen","Name_Abrahamsson, Mr. Abraham August Johannes","Name_Abrahim, Mrs. Joseph (Sophie Halaut Easu)","Name_Aks, Master. Philip Frank",...,Ticket_SOTON/OQ 392083,Ticket_STON/O 2. 3101268,Ticket_STON/O 2. 3101291,Ticket_STON/O2. 3101270,Ticket_STON/OQ. 369943,Ticket_W./C. 14260,Ticket_W./C. 14266,Ticket_W./C. 6607,Ticket_W./C. 6608,Ticket_W.E.P. 5734
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,...,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,1100.500000,2.265550,0.447368,0.392344,0.002392,0.002392,0.002392,0.002392,0.002392,0.002392,...,0.002392,0.002392,0.002392,0.002392,0.002392,0.002392,0.002392,0.004785,0.002392,0.002392
std,120.810458,0.841838,0.896760,0.981429,0.048912,0.048912,0.048912,0.048912,0.048912,0.048912,...,0.048912,0.048912,0.048912,0.048912,0.048912,0.048912,0.048912,0.069088,0.048912,0.048912
min,892.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1100.500000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1204.750000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1309.000000,3.000000,8.000000,9.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [37]:
y.value_counts()

Survived    270
Perished    102
Missing      46
Name: Embarked, dtype: int64

In [38]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, y, random_state=1)

# *SCALING DATA*

In [39]:
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()
X_scaler= scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# LOGISTIC REGRESSION MODEL

In [40]:
#logistic regression model
from sklearn.linear_model import LogisticRegression
classifier= LogisticRegression(solver='lbfgs', random_state=1)
classifier

LogisticRegression(random_state=1)

In [41]:
classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [42]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.8178913738019169
Testing Data Score: 0.6190476190476191


In [43]:
predictions= classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
358,Survived,Missing
164,Survived,Survived
17,Survived,Perished
67,Perished,Survived
4,Survived,Survived
377,Survived,Survived
214,Survived,Survived
290,Perished,Survived
381,Survived,Missing
5,Survived,Survived


In [44]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

     Missing       0.50      0.09      0.15        11
    Perished       0.21      0.14      0.17        21
    Survived       0.69      0.84      0.75        73

    accuracy                           0.62       105
   macro avg       0.47      0.36      0.36       105
weighted avg       0.57      0.62      0.57       105



# DECISION TREE

In [45]:
from sklearn import tree
model= tree.DecisionTreeClassifier()
model= model.fit(X_train_scaled, y_train)

In [49]:
predictions= model.predict(X_test_scaled)


In [55]:
cm= confusion_matrix(y_test, predictions)
cm

array([[ 2,  1,  8],
       [ 1,  8, 12],
       [ 2, 16, 55]])

In [51]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

     Missing       0.40      0.18      0.25        11
    Perished       0.32      0.38      0.35        21
    Survived       0.73      0.75      0.74        73

    accuracy                           0.62       105
   macro avg       0.48      0.44      0.45       105
weighted avg       0.62      0.62      0.61       105

